In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [5]:
def leer_csv(ruta):
    return pd.read_csv(ruta)

In [16]:
TEST_SIZE = 0.1
RANDOM_STATE = 42

def separar_train_test(df_train, logaritmico):
    X = df_train.copy().drop(columns = ['precio', 'log_precio'])
    if logaritmico:
        y = df_train['log_precio']
    else:
        y = df_train['precio']
    return train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)
    

In [17]:
RUTA_TRAIN = 'data/train_preproc.csv'
RUTA_TEST = 'data/test_preproc.csv'

def entrenar(modelos, nombres_modelos, logaritmico, predecir_test = False):

    df_train = leer_csv(RUTA_TRAIN)

    X_train, X_test, Y_train, Y_test = separar_train_test(df_train, logaritmico)
    
    resultado = []
    
    for nombre,modelo in zip(nombres_modelos, modelos):
        print('------------------------------------------------')
        print('Comienza a entrenar: '+ nombre)
        print()
        
        modelo = modelo   
        modelo.fit(X_train, Y_train)
        prediccion = modelo.predict(X_test)
        
        if logaritmico:
            score = mean_absolute_error(np.exp(prediccion), np.exp(Y_test))
        else:
            score = mean_absolute_error(prediccion, Y_test)
            
        resultado.append((str(nombre), str(score)))

        print('**FINALIZADO** ' + nombre + ' Score: ' + str(score))
        print()
    return resultado, modelo



In [18]:
CON_PRECIO_LOGARITMICO = True

#MODELOS = [RandomForestRegressor(n_estimators = 200, max_depth = 500, n_jobs = -1), XGBRegressor(n_estimators=200,max_depth=20,subsample=0.7,min_child_weight =3,colsample_bytree = 0.7,eta = 0.1), BaggingRegressor()]
MODELOS = [XGBRegressor(n_estimators=200,max_depth=20,subsample=0.7,min_child_weight =3,colsample_bytree = 0.7,eta = 0.1)]
#NOMBRES_MODELOS = ['RandomForestRegressor', 'XGBoost', 'Bagging']
NOMBRES_MODELOS = ['XGBoost']
resultado ,modelo= entrenar(MODELOS, NOMBRES_MODELOS, CON_PRECIO_LOGARITMICO)

------------------------------------------------
Comienza a entrenar: XGBoost

[23:14:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
**FINALIZADO** XGBoost Score: 487797.2694374844



**FINALIZADO** XGBoost Score: 543519.3351068652

**FINALIZADO** XGBoost Score: 543808.3471162983 //Antes de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 500191.9174657849 //Despues de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 501766.49058353336 //Despues de agregar gimnasio, usos multiples y piscina

**FINALIZADO** XGBoost Score: 500862.1657935754 // Agrego solo piscina.

**FINALIZADO** XGBoost Score: 499181.4921723928 //Agrego precio promedio por m2 por provincia. Dejando piscina mejora.

**FINALIZADO** XGBoost Score: 496056.96402535716 //Tony agrega lat+lng precio promedio por m2

**FINALIZADO** XGBoost Score: 495398.1192696873 //Agrego feature tiene_sup_descubierta

**FINALIZADO** XGBoost Score: 491674.73406999116 //Agrego feature diff_metros_totales y metros_cubiertos

**FINALIZADO** XGBoost Score: 491335.880882687 //Agrego one hot encoding para antiguedad

**FINALIZADO** XGBoost Score: 490347.0285999908 //Dejando columna antiguedad Y el one hot encoding (antes se dropeaba)

**FINALIZADO** XGBoost Score: 489407.5245960651 //No imputando. Poniendo listas vacias

**FINALIZADO** XGBoost Score: 487797.2694374844 // **ENTREGA** USANDO 0.1 DE TEST_SIZE

In [20]:
def predecir(indice_modelo, guardar_csv = True):
    
    df_train = leer_csv(RUTA_TRAIN).drop(columns = ['precio', 'log_precio'])
    to_predict = leer_csv(RUTA_TEST)
    
    df_test_original = leer_csv('data/test.csv')
    Id = df_test_original['id']
    
    columnas_train = df_train.columns
    to_predict = to_predict.reindex(columns = columnas_train)
    
    predicciones = modelo.predict(to_predict)
    
    df_resultado = pd.DataFrame()
    df_resultado['Id'] = Id
    df_resultado['target'] = np.exp(predicciones)
    if guardar_csv:
        df_resultado.to_csv('resultados2.csv',index=False)
        
    return df_resultado

INDICE_MODELO = 0

df_resultado = predecir(modelo)

In [21]:
df_resultado

,Id,target
0,4941,6.974868e+06
1,51775,9.450501e+05
2,115253,2.341713e+06
3,299321,1.625022e+06
4,173570,7.189048e+05
...,...,...
59995,75094,3.670187e+06
59996,171847,8.199179e+05
59997,138313,7.447660e+05
59998,271268,1.616250e+06
